In [1]:
# import all the libraries

# Data exploration and analysis tools
import pandas as pd
import seaborn as sns
import numpy as np
from ast import literal_eval
import re as re

In [2]:
survey_data = pd.read_csv('San_Francisco_City_Survey_Data_1996-2017.csv')

In [3]:
survey_df = pd.DataFrame(data=survey_data)

In [4]:
total_columns = survey_df.columns
print(len(total_columns))
print(survey_df.columns.nunique())

92
92


In [5]:
# Data Dictionary for survey data
column_names = {
    'id':'Unique id',
    'year':'Survey year',
    'mode':'survey mode',
    'language':'survey language',
    'dlivedsf':'Length of SF residence 1996-2009 (Groupings change in 2011)', #Made Contiguous 
    'primlang_1':'primary language 1',
    'primlang_2':'primary language',
    'primlang_3':'primary language',
    'primlang_4':'primary_language',
    'dage':'Respondents age group (Age groups change in 2011, 2017)', #Made Contiguous 
    'dethnic':'Respondents ethnicity',
    'mixed_1':'mixed race or ethnics',
    'mixed_2':'mixed race or ethnics',
    'mixed_3':'mixed race or ethnics',
    'mixed_4':'mixed race or ethnics',
    'deduc':'Respondents highest education completed',
    'dincome':'Household income year prior to survey', #Made Contiguous 
    'dhouse':'Number of people in household', #Made Contiguous 
    'ownrenhm':'Own or rent home',
    'gender':'Respondents sex',
    'dsexornt':'Respondents sexual orientation',
    'zipcode':'zipcode',
    'district':'Supervisorial District',
    'movesf':'Likelihood of moving away from SF in the next 3 years',
    'disablephys':'physically disabled',
    'disablement':'mentally disabled'    
}

mode_dict={
    1:'phone',
    2:'mail',
    3:'web/phone',
    4:'web/mail'
}

language_dict={
    1:'English',
    2:'Spanish',
    3:'Chinese',
    4:'Tagalog'
}

dlivedsf_dict={
    1:2,
    2:5,
    3:10,
    4:20,
    5:30,
    6:40, #30+; I gave this value an extra subjective weight
    7:None
}

dage_dict={
    1:24,
    2:34,
    3:44,
    4:54,
    5:59,
    6:64,
    7:65, #65+
    8:None
}

dethnic_dict={
    1:'Black/African American',
    2:'Asian or Pacific Islander',
    3:'Latino/Hispanic',
    4:'Native American/Indian',
    5:'White/Caucasian',
    6:'Other',
    7:'Mixed Ethnicity',
    8:'Dont know',
    9:None,
    10:'Pacific Islander',
    11:'Arab / Middle Eastern /North African ( 2015 Only); Arab,Middle Eastern, South Asian (2017)',
    12:'Mixed Unspecified',
    13:'Caribbean (2017)'
}

deduc_dict={
    1:'Less than high school',
    2:'High school',
    3:'Less than 4 years of college',
    4:'4 or more years of college/Post Graduate',
    5:None,
}

dincome_dict={
    1:10000,
    2:25000,
    3:35000,
    4:50000,
    5:100000,
    6:200000,
    7:300000, #30000 +
    8:None
}

dhouse_dict={
    1:1,
    2:2,
    3:3,
    4:4,
    5:5,
    6:6, #6 ore more
    7:None,
}

gender_dict={
    1:'Female',
    2:'Male',
    3:'Other',
    4:None,
}

movesf_dict={
    1:'Very likely',
    2:'Somewhat likely',
    3:'Not too likely',
    4:'Not at all likely',
    5:None
}

In [6]:
survey_info = ['id','year','mode','language']
demographics = ['dlivedsf','dage','dethnic','deduc','dincome','dhouse','gender','zipcode','movesf']
active_columns = survey_info + demographics

print(len(survey_info))
print(len(demographics))
print(len(active_columns))

print(active_columns)

4
9
13
['id', 'year', 'mode', 'language', 'dlivedsf', 'dage', 'dethnic', 'deduc', 'dincome', 'dhouse', 'gender', 'zipcode', 'movesf']


In [7]:
discard_columns = []

for t in total_columns:
    if t not in active_columns:
        discard_columns.append(t)


In [8]:
print(discard_columns)

['finweigh', 'general', 'stpvnbrd', 'swcndnbrd', 'cityligh', 'wtrswr', 'water', 'sewer', 'swclnbrd', 'stclnbrd', 'stswc', 'recparsy', 'parkvis', 'athfield', 'recpart', 'parkfa', 'recpract', 'parkgr', 'parklp', 'parkcl', 'libsystm', 'libmai', 'libbra', 'libonlin', 'libbo', 'onlibsvc', 'netlibsvc', 'libsta', 'conmalib', 'connelib', 'trspwlk', 'trsppub', 'trspbike', 'trsptaxi', 'trspdaln', 'trspcpl', 'trspptns', 'trspuber', 'munovr', 'mun12mth', 'muntim', 'muncle', 'munsaf', 'munmgmtcro', 'muncou', 'safdnb', 'safnnb', 'hrd311', 'cont311', 'use311sv', 'info311', 'infowebm', 'svc311', 'svcwebmo', 'phone311', 'web311', 'kids', 'k0to5', 'k6to18', 'kprisch', 'kpubsch', 'knosch', 'kqschl', 'ownrenhm', 'dsexornt', 'primlang_1', 'primlang_2', 'primlang_3', 'primlang_4', 'disablephys', 'disablement', 'district', 'mixed_1', 'mixed_2', 'mixed_3', 'mixed_4', 'issue1', 'issue2', 'issue3']


In [9]:
survey_df_clean = survey_df.copy()
survey_df_clean = survey_df_clean.drop(columns=discard_columns)
survey_df_clean = survey_df_clean[survey_df_clean['year'].isin(['2018','2017','2016','2015'])]

In [10]:
survey_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4345 entries, 16699 to 37971
Data columns (total 13 columns):
id          4345 non-null int64
year        4345 non-null int64
mode        4345 non-null float64
language    4345 non-null float64
dhouse      4345 non-null float64
dlivedsf    4345 non-null float64
movesf      4345 non-null float64
dincome     4345 non-null float64
dage        4345 non-null float64
gender      4345 non-null float64
dethnic     4345 non-null float64
deduc       4345 non-null float64
zipcode     2166 non-null float64
dtypes: float64(11), int64(2)
memory usage: 475.2 KB


In [11]:
survey_df_clean_null = survey_df_clean[survey_df_clean.isnull().any(axis=1)]
survey_df_clean_value = survey_df_clean.copy()
survey_df_clean_value = survey_df_clean_value.dropna()


In [12]:
survey_df_clean_null['year'].unique()

array([2015])

In [13]:
survey_df_clean_value['year'].unique()

array([2017])

In [14]:
survey_df_clean_value['zipcode'].nunique()

31

In [15]:
# I can only use 2017 survey data

survey_df_clean_value.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2166 entries, 16699 to 37971
Data columns (total 13 columns):
id          2166 non-null int64
year        2166 non-null int64
mode        2166 non-null float64
language    2166 non-null float64
dhouse      2166 non-null float64
dlivedsf    2166 non-null float64
movesf      2166 non-null float64
dincome     2166 non-null float64
dage        2166 non-null float64
gender      2166 non-null float64
dethnic     2166 non-null float64
deduc       2166 non-null float64
zipcode     2166 non-null float64
dtypes: float64(11), int64(2)
memory usage: 236.9 KB


In [16]:
survey_df_clean_value_a= survey_df_clean_value.copy()
survey_df_clean_value_a['mode'] = survey_df_clean_value_a['mode'].map(mode_dict)
survey_df_clean_value_a['language'] = survey_df_clean_value_a['language'].map(language_dict)
survey_df_clean_value_a['dhouse'] = survey_df_clean_value_a['dhouse'].map(dhouse_dict)
survey_df_clean_value_a['dlivedsf'] = survey_df_clean_value_a['dlivedsf'].map(dlivedsf_dict)
survey_df_clean_value_a['movesf'] = survey_df_clean_value_a['movesf'].map(movesf_dict)
survey_df_clean_value_a['dincome'] = survey_df_clean_value_a['dincome'].map(dincome_dict)
survey_df_clean_value_a['dage'] = survey_df_clean_value_a['dage'].map(dage_dict)
survey_df_clean_value_a['gender'] = survey_df_clean_value_a['gender'].map(gender_dict)
survey_df_clean_value_a['dethnic'] = survey_df_clean_value_a['dethnic'].map(dethnic_dict)
survey_df_clean_value_a['deduc'] = survey_df_clean_value_a['deduc'].map(deduc_dict)
survey_df_clean_value_a



,id,year,mode,language,dhouse,dlivedsf,movesf,dincome,dage,gender,dethnic,deduc,zipcode
16699,201711681,2017,phone,Chinese,1.0,10.0,Somewhat likely,25000.0,34.0,Female,Asian or Pacific Islander,High school,94114.0
18495,201711805,2017,phone,English,2.0,40.0,Not at all likely,300000.0,65.0,Male,White/Caucasian,4 or more years of college/Post Graduate,94124.0
18885,201711881,2017,phone,English,2.0,40.0,Not at all likely,35000.0,54.0,Male,Black/African American,Less than 4 years of college,94115.0
20949,201711908,2017,phone,English,2.0,20.0,Somewhat likely,200000.0,44.0,Female,Native American/Indian,4 or more years of college/Post Graduate,94110.0
29172,201710361,2017,phone,English,2.0,10.0,Not at all likely,NaN,54.0,Female,None,4 or more years of college/Post Graduate,94132.0
29328,201710504,2017,phone,English,3.0,40.0,Not at all likely,200000.0,59.0,Female,Native American/Indian,4 or more years of college/Post Graduate,94132.0
29376,201710604,2017,phone,English,4.0,20.0,Not at all likely,35000.0,54.0,Female,Asian or Pacific Islander,4 or more years of college/Post Graduate,94122.0
29400,201710605,2017,phone,Chinese,6.0,10.0,Not at all likely,50000.0,59.0,Male,Asian or Pacific Islander,High school,94122.0
29472,201710606,2017,phone,Spanish,2.0,40.0,Not at all likely,35000.0,65.0,Female,White/Caucasian,High school,94112.0
29496,201710607,2017,phone,English,NaN,40.0,Not at all likely,NaN,59.0,Female,None,4 or more years of college/Post Graduate,94131.0


In [17]:
survey_df_clean_value_a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2166 entries, 16699 to 37971
Data columns (total 13 columns):
id          2166 non-null int64
year        2166 non-null int64
mode        2166 non-null object
language    2166 non-null object
dhouse      2145 non-null float64
dlivedsf    2159 non-null float64
movesf      2137 non-null object
dincome     1781 non-null float64
dage        2113 non-null float64
gender      2133 non-null object
dethnic     2066 non-null object
deduc       2124 non-null object
zipcode     2166 non-null float64
dtypes: float64(5), int64(2), object(6)
memory usage: 236.9+ KB


In [18]:
survey_df_clean_value_a_dummies = pd.get_dummies(survey_df_clean_value_a)

In [23]:
survey_df_final = survey_df_clean_value_a_dummies.copy()
survey_df_final = survey_df_final.groupby('zipcode').mean()

In [24]:
survey_df_final

,id,year,dhouse,dlivedsf,dincome,dage,mode_phone,mode_web/phone,language_Chinese,language_English,...,dethnic_Latino/Hispanic,dethnic_Mixed Ethnicity,dethnic_Native American/Indian,dethnic_Other,dethnic_Pacific Islander,dethnic_White/Caucasian,deduc_4 or more years of college/Post Graduate,deduc_High school,deduc_Less than 4 years of college,deduc_Less than high school
zipcode,,,,,,,,,,,,,,,,,,,,,
94102.0,2.017112e+08,2017.0,2.246914,22.951220,77323.943662,48.691358,1.000000,0.000000,0.073171,0.902439,...,0.000000,0.024390,0.378049,0.000000,0.097561,0.109756,0.414634,0.243902,0.231707,0.085366
94103.0,2.017110e+08,2017.0,2.294872,21.076923,111805.555556,48.423077,0.987179,0.012821,0.012821,0.974359,...,0.025641,0.000000,0.435897,0.000000,0.102564,0.205128,0.538462,0.179487,0.256410,0.025641
94104.0,2.017112e+08,2017.0,2.545455,10.727273,113500.000000,40.250000,1.000000,0.000000,0.083333,0.750000,...,0.083333,0.000000,0.333333,0.000000,0.000000,0.250000,0.583333,0.333333,0.000000,0.083333
94105.0,2.017110e+08,2017.0,1.944444,14.555556,189666.666667,45.833333,0.944444,0.055556,0.000000,1.000000,...,0.055556,0.000000,0.500000,0.000000,0.055556,0.000000,0.944444,0.055556,0.000000,0.000000
94107.0,2.017111e+08,2017.0,2.392157,26.450980,156914.893617,50.620000,0.960784,0.039216,0.000000,1.000000,...,0.019608,0.019608,0.588235,0.000000,0.078431,0.058824,0.647059,0.117647,0.196078,0.039216
94108.0,2.017111e+08,2017.0,2.590909,11.826087,95000.000000,42.304348,1.000000,0.000000,0.130435,0.826087,...,0.043478,0.000000,0.260870,0.000000,0.000000,0.086957,0.565217,0.130435,0.217391,0.086957
94109.0,2.017111e+08,2017.0,2.093023,22.000000,123285.714286,51.166667,0.984496,0.015504,0.038760,0.945736,...,0.038760,0.031008,0.480620,0.007752,0.069767,0.069767,0.666667,0.108527,0.147287,0.069767
94110.0,2.017111e+08,2017.0,2.822222,24.805556,148838.709677,49.096045,0.994444,0.005556,0.000000,0.866667,...,0.027778,0.005556,0.500000,0.000000,0.050000,0.327778,0.638889,0.100000,0.161111,0.083333
94111.0,2.017107e+08,2017.0,1.818182,15.181818,131111.111111,51.909091,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.545455,0.000000,0.000000,0.090909,0.727273,0.090909,0.181818,0.000000


In [26]:
survey_df_final.to_csv(path_or_buf='/Users/lizchan/ds_foundations/final_project/survey_clean_2017.csv')